In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/alzheimers-audio/VBSDdataset/VBSDdataset/imagetrain',
    labels="inferred",
    label_mode="binary",
    color_mode="rgb",
    batch_size=32,
    image_size=(295, 295),
    shuffle=True,
    validation_split=0.2,
    subset="training",
    seed=54,
    interpolation="bilinear",
    follow_links=False,
)
test = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/alzheimers-audio/VBSDdataset/VBSDdataset/imagetrain',
    labels="inferred",
    label_mode="binary",
    color_mode="rgb",
    batch_size=32,
    image_size=(295, 295),
    shuffle=True,
    validation_split=0.2,
    subset="validation",
    seed=54,
    interpolation="bilinear",
    follow_links=False,
)
train

In [ ]:
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

In [ ]:
train = (
    train
    .map(convert_to_float)
)
test = (
    test
    .map(convert_to_float)
)

# Keras CNN Model

In [ ]:

from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation="sigmoid", input_shape=(295, 295, 3)),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D(32, (3, 3), activation="sigmoid", padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.20),
    
    keras.layers.Conv2D(32, (3, 3), activation="sigmoid", padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),

    keras.layers.Flatten(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation="sigmoid"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.summary()


In [ ]:
from keras.optimizers import SGD
model.compile(
    optimizer=SGD(lr=0.01),
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

history = model.fit(
    train,
    validation_data=test,
    epochs=20,
)

In [ ]:
print(history.history.keys())
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# SciKit RegressionCV Model

In [2]:
import os, cv2, itertools
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
!pip install tensorflow.io
import tensorflow_io as tfio

     |████████████████████████████████| 22.7 MB 19.6 MB/s eta 0:00:01
     |████████████████████████████████| 454.3 MB 14 kB/s s eta 0:00:01    |█▍                              | 20.0 MB 38.1 MB/s eta 0:00:12     |██▏                             | 31.5 MB 10.9 MB/s eta 0:00:39     |██▍                             | 34.6 MB 10.9 MB/s eta 0:00:39     |████▉                           | 69.3 MB 39.3 MB/s eta 0:00:10     |███████▏                        | 101.4 MB 39.3 MB/s eta 0:00:09     |█████████▌                      | 135.3 MB 44.1 MB/s eta 0:00:08     |█████████████████████▉          | 309.4 MB 44.6 MB/s eta 0:00:04
     |████████████████████████████████| 2.3 MB 52.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 35.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 46.9 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 47.1 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 39.0 MB/s eta 0:00:01
     |███████████████████████

In [3]:
VBSD_dir1 = '../input/alzheimers-audio/VBSDdataset/VBSDdataset/imagetrain/0/' # Directory for training set
VBSD_dir2 = '../input/alzheimers-audio/VBSDdataset/VBSDdataset/imagetrain/1/' # Directory for training set
DemCare_dir1 = '../input/alzheimers-audio/DemCare_imagedata/imagedata/imagetrain/0/' # Directory for testing set
DemCare_dir2 = '../input/alzheimers-audio/DemCare_imagedata/imagedata/imagetrain/1/' # Directory for testing set

In [12]:
import matplotlib.pyplot as plt
VBSD_train_images1 = [VBSD_dir1+i for i in os.listdir(VBSD_dir1)] 
VBSD_train_images2 = [VBSD_dir2+i for i in os.listdir(VBSD_dir2)]
DemCare_train_images1 =  [DemCare_dir1+i for i in os.listdir(DemCare_dir1)]
DemCare_train_images2 =  [DemCare_dir2+i for i in os.listdir(DemCare_dir2)]
#print(train_images)
def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR)
   # print(file_path)
   # print(img.shape)
    imgg = cv2.resize(img, (295, 295), interpolation=cv2.INTER_CUBIC)
    time_mask = tfio.audio.time_mask(imgg, param=10)
    #freq_mask = tfio.audio.freq_mask(img, param=10)
    return imgg, time_mask#, freq_mask
def read_image_noaugment(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR)
    #print(img.shape)
   # print(file_path)
    imgg = cv2.resize(img, (295, 295), interpolation=cv2.INTER_CUBIC)
    return imgg


def prep_data(images, val):    
    m = len(images) * 2
    n_x = 295 * 295 * 3
    
    X = np.ndarray((n_x, m), dtype=np.uint8)
    y = np.zeros((1, m))
    print ("X shape is {}".format(X.shape))
    
    for i, image_file in enumerate(images):
        #print(image_file)
        f = i * 2
        image, image_time_mask = read_image(image_file)
        X[:, f] = np.squeeze(image.reshape((n_x, 1)))   
        X[:, f + 1] = np.squeeze(image.reshape((n_x, 1))) 
        if val == 0:
            y[0, f] = 0
            y[0, f + 1] = 0
        elif val == 1:
            y[0, f] = 1
            y[0, f + 1] = 1
        if f%50 == 0: print('Processed {} of {}'.format(f, m))
    
    return X.transpose(), y.flatten()

def prep_data_noaugment(images, val):    
    m = len(images)
    n_x = 295 * 295 * 3
    
    X = np.ndarray((n_x, m), dtype=np.uint8)
    y = np.zeros((1, m))
    print ("X shape is {}".format(X.shape))
    
    for i, image_file in enumerate(images):
        #print(image_file)
        image = read_image_noaugment(image_file)

        X[:, i] = np.squeeze(image.reshape((n_x, 1)))   
        if val == 0:
            y[0, i] = 0
        elif val == 1:
            y[0,  i] = 1
        if i%50 == 0: print('Processed {} of {}'.format(i, m))
    
    return X.transpose(), y.flatten()



In [13]:
X_train1, y_train1 = prep_data(VBSD_train_images1, 0)
X_train2, y_train2 = prep_data(VBSD_train_images2, 1)
X_t = np.concatenate((X_train1, X_train2),axis = 0)
y_t = np.concatenate([y_train1, y_train2])
X_train, X_test, y_train, y_test = train_test_split(X_t, y_t, test_size=0.20, random_state=42)

X_newtrain1, y_newtrain1 = prep_data_noaugment(DemCare_train_images1, 0)
X_newtrain2, y_newtrain2 = prep_data_noaugment(DemCare_train_images2, 1)
X_newtrain = np.concatenate((X_newtrain1, X_newtrain2),axis = 0)
y_newtrain = np.concatenate([y_newtrain1, y_newtrain2])



X shape is (261075, 500)
Processed 0 of 500
Processed 50 of 500
Processed 100 of 500
Processed 150 of 500
Processed 200 of 500
Processed 250 of 500
Processed 300 of 500
Processed 350 of 500
Processed 400 of 500
Processed 450 of 500
X shape is (261075, 508)
Processed 0 of 508
Processed 50 of 508
Processed 100 of 508
Processed 150 of 508
Processed 200 of 508
Processed 250 of 508
Processed 300 of 508
Processed 350 of 508
Processed 400 of 508
Processed 450 of 508
Processed 500 of 508
X shape is (261075, 231)
Processed 0 of 231
Processed 50 of 231
Processed 100 of 231
Processed 150 of 231
Processed 200 of 231
X shape is (261075, 254)
Processed 0 of 254
Processed 50 of 254
Processed 100 of 254
Processed 150 of 254
Processed 200 of 254
Processed 250 of 254


In [14]:
from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(cv=5, random_state=0, max_iter = 2000).fit(X_train, y_train)

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=0, max_iter=1000, tol=1e-3).fit(X_train, y_train)

In [15]:
def test(X_test, y_test, clf):
    correct = 0
    total = 0
    num_rows, num_col = X_test.shape
    for i in range(num_rows):
        x = X_test[i].reshape(1,-1)
        #print(x.shape)
        y = y_test[i]
        y_hat = clf.predict(x)
        if y_hat == y:
            correct += 1 
        total += 1
    #print(correct, " / ", total)
    print("Model was", str(correct*100/total) + "% correct")

In [16]:
test(X_test, y_test, clf)
#test(X_newtrain2, y_newtrain2, clf)
#test(X_test, y_test, sgd_clf)

Model was 97.02970297029702% correct
